In [ ]:
max_steps = 20_000 #after 20k steps, agent will stop trying
## Our Grid (starts from zero)
minX=0
minY=0
maxX=19
maxY=19

In [ ]:
3<4<5

In [ ]:
#this is test using velocity to reach the target not rudder


import gym
import numpy as np

class SailingEnv(gym.Env):
    def __init__(self):
        self.maxsteps=max_steps
        self.minX=minX
        self.maxX=maxX
        self.minY=minY
        self.maxY=maxY
        self.target = np.array([8, 15], dtype=np.float64)        
        self.action_space = gym.spaces.Discrete(3)
        self.current_pos = np.array([0, 0], dtype=np.float64)
        self.current_vel = np.array([0, 0], dtype=np.float64)
        #self.observation_space = gym.spaces.MultiDiscrete([10, 10])
        #self.reset()

    def reset(self):
        self.current_pos = np.array([0, 0])        
        self.current_vel = np.array([0, 0])
        self.combined=np.concatenate((self.current_pos, self.current_vel)) 
        return tuple(self.combined.reshape(1,-1)[0])

        
    
    
    def step(self, action):
#         # Define velocity vector((s))
         velocity_vectors = np.array([[1, 0], [1, 1], [1, -1]])
    
#         # Set the rudder angle based on the action
         self.velocity = velocity_vectors[action]

         self.current_pos = self.current_pos + self.velocity * 1
         reward = -np.sqrt(np.sum(np.square(self.current_pos - self.target))) #reward me vash apostash
         done = np.allclose(self.current_pos, self.target) #epistrefei True otan ftasei sto target
         truncated = not (self.minX<self.current_pos[0]<self.maxX and self.minY<self.current_pos[1]<self.maxY)
         #info = {'current_pos': self.current_pos, 'reward': reward, 'done': done}
         self.combined=np.concatenate((self.current_pos, self.velocity))
        
         return tuple(self.combined.reshape(1,-1)[0]), reward, done, truncated
         #return info
        #return self._get_obs(), reward, done, info

#     def _get_obs(self):
#         return tuple(self.current_pos.astype(np.int64) + 5)


env = SailingEnv()

In [ ]:
q_table = np.random.uniform(low=-2, high=0, size=([maxX+1,maxY+1,maxX+1,maxY+1] + [env.action_space.n]))

In [ ]:


# Set learning parameters
LEARNING_RATE = 0.0001
DISCOUNT = 0.95
EPSILON=0.9

# Q-learning algorithm
num_episodes = 1000
alpha = 0.001
gamma = 0.9
epsilon = 0.1




# Set exploration parameters
#exploration_rate = 1.0
#max_exploration_rate = 1.0
#min_exploration_rate = 0.01
#exploration_decay_rate = 0.001


In [ ]:
success=0
for i in range(num_episodes):
    done = False
    episode_reward = 0
    current_state = env.reset()
    
    while not done:
        if np.random.random() < EPSILON:
            # take a random action with probability epsilon
            action = env.action_space.sample()
        else:
            # choose the action with the highest Q-value
            action = np.argmax(q_table[current_state])
        
        next_observation, reward, done, truncated = env.step(action)
        
        if truncated:
            #poly mikro Q value otan o agent odhgeitai eksw apo to grid
            #gia na mhn ksanaepileksei tetoia action otan vrethei sthn idia thesh
            q_table[current_state+(action,)]=-20
            break
        else:
            #Q_max
            Q_max = np.max(q_table[next_observation])
            #Q_current
            Q_current = q_table[current_state + (action,)]
            #new_q
            new_q = (1-LEARNING_RATE)*Q_current + LEARNING_RATE*(reward+DISCOUNT*Q_max)
            #update q_table
            q_table[current_state+(action,)]=new_q
            #replace new_state as current_state
            
        #print(done)
        if done:
            success+=1
            #print("eftasa")

    
        current_state = next_observation
        episode_reward += reward

print(success)